# 1. Install and Import Dependencies

In [178]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import StandardScaler, LabelEncoder
le = LabelEncoder()
import pickle

# std = StandardScaler()

# 2. Load the data

In [179]:
df = pd.read_csv("datasets/Crop_recommendation.csv") 

# 3. Pre-processing the data

In [180]:
x = df.drop(columns="label")
y = pd.DataFrame(df['label'],columns=['label'])
x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=0, train_size = .75)
print(y_train)
y_train["label"] = le.fit_transform(y_train["label"])
# 
y_test["label"] = le.transform(y_test["label"])
# y_test

          label
1975     cotton
1287     grapes
760   blackgram
267    chickpea
2007       jute
...         ...
1033     banana
1731     papaya
763   blackgram
835      lentil
1653     orange

[1650 rows x 1 columns]


In [181]:
x_train

,N,P,K,temperature,humidity,ph,rainfall
1975,134,56,18,23.808346,83.919026,6.691268,70.973583
1287,29,122,196,41.948657,81.155952,5.638328,73.068630
760,25,68,19,29.399827,64.255107,7.108450,67.476773
267,41,69,82,20.023815,16.632945,6.715587,68.978065
2007,70,43,40,24.355641,88.803910,6.176860,169.116803
...,...,...,...,...,...,...,...
1033,102,71,48,28.654563,79.286937,5.695268,102.463378
1731,34,68,51,27.347349,94.177567,6.687088,40.351531
763,35,64,15,28.474423,63.536045,6.500145,69.527441
835,39,65,23,25.434598,69.126134,7.685959,41.026829


In [182]:
y_train

,label
1975,6
1287,7
760,2
267,3
2007,8
...,...
1033,1
1731,17
763,2
835,10


In [183]:
y_train = to_categorical(y_train, 22)
y_test = to_categorical(y_test, 22)
y_train

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

# 4. Model

In [184]:
def crop_predection__model():

    # Define the model
    model = tf.keras.models.Sequential([ 
      tf.keras.layers.Dense(32 , activation='relu'),
      tf.keras.layers.Dense(64 , activation='relu'),
      tf.keras.layers.Dense(22 , activation='softmax'),
  
    ]) 


    # Compile the model
    model.compile(optimizer='adam', 
                  loss='categorical_crossentropy', 
                  metrics=['accuracy']) 
        
    return model

In [185]:
model =crop_predection__model()

In [186]:
history = model.fit(x_train, y_train, epochs=5)

Epoch 1/5
52/52 [==============================] - 0s 1ms/step - loss: 12.2606 - accuracy: 0.1958
Epoch 2/5
52/52 [==============================] - 0s 1ms/step - loss: 1.2622 - accuracy: 0.6188
Epoch 3/5
52/52 [==============================] - 0s 1ms/step - loss: 0.5354 - accuracy: 0.8152
Epoch 4/5
52/52 [==============================] - 0s 1ms/step - loss: 0.3713 - accuracy: 0.8770
Epoch 5/5
52/52 [==============================] - 0s 1ms/step - loss: 0.2941 - accuracy: 0.8952


# 5. Testing

In [187]:
model.evaluate(x_test, y_test, batch_size=50)  

11/11 [==============================] - 0s 1ms/step - loss: 0.2767 - accuracy: 0.9018


[0.2766963839530945, 0.9018181562423706]

In [195]:
pred = model.predict(x_train.iloc[[0]])
pred
# x_train.iloc[[0]]

1/1 [==============================] - 0s 15ms/step


array([[1.5257436e-06, 8.5085063e-05, 7.0057141e-08, 1.7297607e-03,
        5.5396876e-10, 2.8317125e-13, 9.9737370e-01, 1.6032589e-12,
        2.4533886e-09, 8.4815471e-10, 2.5528854e-07, 7.5111841e-04,
        4.8491217e-10, 4.5035173e-07, 1.5769366e-09, 5.8944281e-09,
        2.7625052e-05, 1.5067218e-06, 2.5680852e-13, 1.9850352e-12,
        5.9428090e-10, 2.8874510e-05]], dtype=float32)

In [189]:
max3_values = np.argsort(pred[0])[::-1]
max3_values

array([ 6,  3, 11,  1, 21, 16,  0, 17, 13, 10,  2, 15,  8, 14,  9, 20,  4,
       12, 19,  7,  5, 18], dtype=int64)

In [190]:
le.inverse_transform(max3_values)

array(['cotton', 'chickpea', 'maize', 'banana', 'watermelon', 'orange',
       'apple', 'papaya', 'mothbeans', 'lentil', 'blackgram', 'muskmelon',
       'jute', 'mungbean', 'kidneybeans', 'rice', 'coconut', 'mango',
       'pomegranate', 'grapes', 'coffee', 'pigeonpeas'], dtype=object)

# Pickle file

In [191]:
# pickle.dump(model, open("model.pkl", "wb"))

In [192]:
# model.save("saved_model/crop_predection__model")

In [193]:
# pickle.dump(model , open("model.pkl" , "wb"))
model.save("my_h5_model.h5")